# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115924e+02     1.556638e+00
 * time: 0.08589506149291992
     1     1.043925e+01     9.034950e-01
 * time: 1.1706740856170654
     2    -1.132779e+01     1.019744e+00
 * time: 1.228572130203247
     3    -3.424671e+01     8.348939e-01
 * time: 1.319404125213623
     4    -4.750148e+01     6.145921e-01
 * time: 1.3876781463623047
     5    -5.693361e+01     2.119975e-01
 * time: 1.473938226699829
     6    -5.981627e+01     1.367954e-01
 * time: 1.5248920917510986
     7    -6.089656e+01     5.583724e-02
 * time: 1.575618028640747
     8    -6.129626e+01     7.500186e-02
 * time: 1.6283950805664062
     9    -6.158818e+01     3.995313e-02
 * time: 1.6896350383758545
    10    -6.179786e+01     3.075629e-02
 * time: 1.7405781745910645
    11    -6.195809e+01     2.323387e-02
 * time: 1.7917580604553223
    12    -6.202612e+01     2.179566e-02
 * time: 1.8489031791687012
    13    -6.209897e+01     1.602252e-02
 * time: 1.9011170864105

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671054
    AtomicLocal         -18.8557637
    AtomicNonlocal      14.8522628
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485356 
    Xc                  -19.3336814

    total               -62.261666458512
